In [1]:
import pandas as pd
import pubchempy as pcp
import rdkit
from rdkit.Chem import rdMolDescriptors
import numpy as np
import pymatgen
from padelpy import from_smiles

In [2]:
df_initial = pd.read_csv(r'C:\Users\wwapp\OneDrive\Рабочий стол\ProgrOfChemTasks\11.csv')

In [3]:
df_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DOI                     5000 non-null   object 
 1   Date                    4573 non-null   object 
 2   Journal                 4573 non-null   object 
 3   Title                   4573 non-null   object 
 4   Name                    4996 non-null   object 
 5   measurement_error       5000 non-null   float64
 6   measurement_wavelength  565 non-null    object 
 7   measurement_method      5000 non-null   object 
 8   normalised_name         2941 non-null   object 
 9   raw_value               5000 non-null   object 
 10  specifier               5000 non-null   object 
dtypes: float64(1), object(10)
memory usage: 429.8+ KB


In [4]:
df_initial.Name

0                   NaCl
1                 NaPh4B
2                   FITC
3                    CuO
4            Propan-1-ol
              ...       
4995                 ZnS
4996    tantalum nitride
4997           i-Butanol
4998         Cyclohexane
4999          Chloroform
Name: Name, Length: 5000, dtype: object

# Let's start our parsing with the PubChemPy API. Using it, we can collect 43 descriptors (their name presented in the list below):

In [5]:
descriptors_pcp = pcp.get_compounds('NaCl', 'name', as_dataframe=True).columns.to_list()
for i in descriptors_pcp:
    print(i)

atom_stereo_count
atoms
bond_stereo_count
bonds
cactvs_fingerprint
canonical_smiles
charge
complexity
conformer_id_3d
conformer_rmsd_3d
coordinate_type
covalent_unit_count
defined_atom_stereo_count
defined_bond_stereo_count
effective_rotor_count_3d
elements
exact_mass
feature_selfoverlap_3d
fingerprint
h_bond_acceptor_count
h_bond_donor_count
heavy_atom_count
inchi
inchikey
isomeric_smiles
isotope_atom_count
iupac_name
mmff94_energy_3d
mmff94_partial_charges_3d
molecular_formula
molecular_weight
monoisotopic_mass
multipoles_3d
pharmacophore_features_3d
record
rotatable_bond_count
shape_fingerprint_3d
shape_selfoverlap_3d
tpsa
undefined_atom_stereo_count
undefined_bond_stereo_count
volume_3d
xlogp


In [6]:
#create a DataFrame that will include all data about descriptors
df_pcp = pd.DataFrame(columns=descriptors_pcp)

In [7]:
# enumeration of all names in the DataFrame. Calculated for this name descriptors is inserting in the df_pcp Dataframe. 
#If this name cannot be used for calculating descriptors, nan row is incerting
df_nan = pd.DataFrame(columns=descriptors_pcp, index=[0])
for i,_ in df_initial.iterrows():
    try:
        df_pcp = pd.concat([df_pcp, pcp.get_compounds(df_initial.Name[i], 'name', as_dataframe=True).iloc[:1]])
    except:
        df_pcp = pd.concat([df_pcp, df_nan])

In [8]:
#let's check everything is OK
df_pcp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 5234 to 6212
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   atom_stereo_count            3445 non-null   object
 1   atoms                        3445 non-null   object
 2   bond_stereo_count            3445 non-null   object
 3   bonds                        3445 non-null   object
 4   cactvs_fingerprint           3445 non-null   object
 5   canonical_smiles             3445 non-null   object
 6   charge                       3445 non-null   object
 7   complexity                   3445 non-null   object
 8   conformer_id_3d              0 non-null      object
 9   conformer_rmsd_3d            0 non-null      object
 10  coordinate_type              3445 non-null   object
 11  covalent_unit_count          3445 non-null   object
 12  defined_atom_stereo_count    3445 non-null   object
 13  defined_bond_stereo_count    3

In [11]:
df_pcp.to_csv(r'C:\Users\wwapp\OneDrive\Рабочий стол\ProgrOfChemTasks\df_pcp.csv', index=False) asas 

In [306]:
df_pcp

,atom_stereo_count,atoms,bond_stereo_count,bonds,cactvs_fingerprint,canonical_smiles,charge,complexity,conformer_id_3d,conformer_rmsd_3d,...,pharmacophore_features_3d,record,rotatable_bond_count,shape_fingerprint_3d,shape_selfoverlap_3d,tpsa,undefined_atom_stereo_count,undefined_bond_stereo_count,volume_3d,xlogp
5234,0,"[{'aid': 1, 'number': 17, 'element': 'Cl', 'x'...",0,"[{'aid1': 1, 'aid2': 2, 'order': 7}]",0000000000000000000000000010000000000100000000...,[Na+].[Cl-],0,2,None,None,...,None,"{'id': {'id': {'cid': 5234}}, 'atoms': {'aid':...",0,None,None,0,0,0,None,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN
18730,0,"[{'aid': 1, 'number': 16, 'element': 'S', 'x':...",0,"[{'aid1': 1, 'aid2': 28, 'order': 2}, {'aid1':...",1100000001111010001110000000000001000000000000...,C1=CC2=C(C=C1N=C=S)C(=O)OC23C4=C(C=C(C=C4)O)OC...,0,668,None,None,...,None,"{'id': {'id': {'cid': 18730}}, 'atoms': {'aid'...",1,None,None,120,0,0,None,4.8
14829,0,"[{'aid': 1, 'number': 29, 'element': 'Cu', 'x'...",0,"[{'aid1': 1, 'aid2': 2, 'order': 2}]",0000000000000000001000000000000000000000000000...,O=[Cu],0,2,None,None,...,None,"{'id': {'id': {'cid': 14829}}, 'atoms': {'aid'...",0,None,None,17.1,0,0,None,None
1031,0,"[{'aid': 1, 'number': 8, 'element': 'O', 'x': ...",0,"[{'aid1': 1, 'aid2': 3, 'order': 1}, {'aid1': ...",1100000001000000001000000000000000000000000000...,CCCO,0,7.2,None,None,...,None,"{'id': {'id': {'cid': 1031}}, 'atoms': {'aid':...",1,None,None,20.2,0,0,None,0.3
8857,0,"[{'aid': 1, 'number': 8, 'element': 'O', 'x': ...",0,"[{'aid1': 1, 'aid2': 3, 'order': 1}, {'aid1': ...",1100000001100000001100000000000000000000000000...,CCOC(=O)C,0,49.5,None,None,...,None,"{'id': {'id': {'cid': 8857}}, 'atoms': {'aid':...",2,None,None,26.3,0,0,None,0.7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,NaN
887,0,"[{'aid': 1, 'number': 8, 'element': 'O', 'x': ...",0,"[{'aid1': 1, 'aid2': 2, 'order': 1}, {'aid1': ...",1000000000000000001000000000000000000000000000...,CO,0,2,None,None,...,None,"{'id': {'id': {'cid': 887}}, 'atoms': {'aid': ...",0,None,None,20.2,0,0,None,-0.5
24261,0,"[{'aid': 1, 'number': 14, 'element': 'Si', 'x'...",0,"[{'aid1': 1, 'aid2': 2, 'order': 2}, {'aid1': ...",0000000000000000001100000000100000000000000000...,O=[Si]=O,0,18.3,None,None,...,None,"{'id': {'id': {'cid': 24261}}, 'atoms': {'aid'...",0,None,None,34.1,0,0,None,None


# Next, we can use rdkit software to collect 43 new descriptors:

In [14]:
descriptors_rdkit = rdMolDescriptors.Properties.GetAvailableProperties()
for i in descriptors_rdkit:
    print(i)

exactmw
amw
lipinskiHBA
lipinskiHBD
NumRotatableBonds
NumHBD
NumHBA
NumHeavyAtoms
NumAtoms
NumHeteroatoms
NumAmideBonds
FractionCSP3
NumRings
NumAromaticRings
NumAliphaticRings
NumSaturatedRings
NumHeterocycles
NumAromaticHeterocycles
NumSaturatedHeterocycles
NumAliphaticHeterocycles
NumSpiroAtoms
NumBridgeheadAtoms
NumAtomStereoCenters
NumUnspecifiedAtomStereoCenters
labuteASA
tpsa
CrippenClogP
CrippenMR
chi0v
chi1v
chi2v
chi3v
chi4v
chi0n
chi1n
chi2n
chi3n
chi4n
hallKierAlpha
kappa1
kappa2
kappa3
Phi


In [15]:
#create a DataFrame that will include all data about descriptors
df_rdkit = pd.DataFrame(columns=descriptors_rdkit)

In [275]:
# enumeration of all names in the DataFrame. Calculated for this name descriptors is inserting in the df_rdkit Dataframe. 
#If this name cannot be used for calculating descriptors, nan row is incerting
df_nan = pd.DataFrame(columns=descriptors_rdkit, index=[0])
get_descriptors = rdMolDescriptors.Properties(descriptors_rdkit)
for i,_ in df_initial.iterrows():
    try:
        compound = pcp.get_compounds(df_initial.Name[i], 'name')
        smile = compound[0].canonical_smiles
        mol = rdkit.Chem.MolFromSmiles(smile)
        df_rdkit = pd.concat([df_rdkit, pd.DataFrame(np.array(get_descriptors.ComputeProperties(mol)).reshape(1,-1), columns=descriptors_rdkit)])
    except:
        df_rdkit = pd.concat([df_rdkit, df_nan])

[12:38:29] WARNING: not removing hydrogen atom without neighbors
[12:42:19] WARNING: not removing hydrogen atom without neighbors
[12:46:32] WARNING: not removing hydrogen atom without neighbors
[12:52:13] WARNING: not removing hydrogen atom without neighbors
[12:52:36] WARNING: not removing hydrogen atom without neighbors
[13:02:34] WARNING: not removing hydrogen atom without neighbors
[13:11:35] WARNING: not removing hydrogen atom without neighbors
[13:12:18] WARNING: not removing hydrogen atom without neighbors
[13:17:15] WARNING: not removing hydrogen atom without neighbors
[13:23:12] WARNING: not removing hydrogen atom without neighbors
[13:23:55] WARNING: not removing hydrogen atom without neighbors


In [276]:
#let's check everything is OK
df_rdkit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 0
Data columns (total 43 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   exactmw                          3445 non-null   float64
 1   amw                              3445 non-null   float64
 2   lipinskiHBA                      3445 non-null   float64
 3   lipinskiHBD                      3445 non-null   float64
 4   NumRotatableBonds                3445 non-null   float64
 5   NumHBD                           3445 non-null   float64
 6   NumHBA                           3445 non-null   float64
 7   NumHeavyAtoms                    3445 non-null   float64
 8   NumAtoms                         3445 non-null   float64
 9   NumHeteroatoms                   3445 non-null   float64
 10  NumAmideBonds                    3445 non-null   float64
 11  FractionCSP3                     3445 non-null   float64
 12  NumRings               

In [12]:
df_rdkit.to_csv(r'C:\Users\wwapp\OneDrive\Рабочий стол\ProgrOfChemTasks\df_rdkit.csv', index=False) as as 

NameError: name 'df_rdkit' is not defined

In [278]:
df_rdkit

,exactmw,amw,lipinskiHBA,lipinskiHBD,NumRotatableBonds,NumHBD,NumHBA,NumHeavyAtoms,NumAtoms,NumHeteroatoms,...,chi0n,chi1n,chi2n,chi3n,chi4n,hallKierAlpha,kappa1,kappa2,kappa3,Phi
0,57.958622,58.443,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,...,1.377964,0.000000,0.000000,0.000000,0.000000,1.290000,10.367820,2.290000,0.065194,11.871154
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,389.035793,389.388,6.0,2.0,1.0,2.0,7.0,28.0,39.0,7.0,...,14.170786,8.333274,4.931788,4.931788,3.815680,-3.670000,16.499791,5.661360,2.113625,3.336116
0,78.924512,79.545,1.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,...,0.709760,0.123091,0.000000,0.000000,0.000000,0.319481,2.319481,1.319481,1.449562,1.530255
0,60.057515,60.096,1.0,1.0,1.0,1.0,1.0,4.0,12.0,1.0,...,2.861427,1.523335,0.223607,0.223607,0.000000,-0.040000,3.960000,2.960000,1.960000,2.930400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,197.992939,198.067,7.0,3.0,4.0,3.0,4.0,12.0,19.0,8.0,...,5.836061,2.808234,0.763725,0.763725,0.387274,-0.910000,11.090000,4.199494,7.200011,3.881032
0,194.951070,194.955,1.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,...,0.894427,0.200000,0.000000,0.000000,0.000000,0.450260,2.450260,1.450260,0.671200,1.776757
0,74.073165,74.123,1.0,1.0,1.0,1.0,1.0,5.0,15.0,1.0,...,3.731671,1.879177,0.365148,0.365148,0.000000,-0.040000,4.960000,2.212525,3.960000,2.194825
0,84.093900,84.162,0.0,0.0,0.0,0.0,0.0,6.0,18.0,0.0,...,4.242641,3.000000,1.500000,1.500000,1.060660,0.000000,4.166667,2.222222,1.000000,1.543210


In [ ]:
import rdkit
from rdkit.ML.Descriptors import Parser
from rdkit.Chem import Descriptors

In [ ]:
def getMolDescriptors(mol, missingVal=None):
    ''' calculate the full list of descriptors for a molecule
    
        missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res

In [ ]:
doravirine = rdkit.Chem.MolFromSmiles('Cn1c(n[nH]c1=O)Cn2ccc(c(c2=O)Oc3cc(cc(c3)Cl)C#N)C(F)(F)F')
getMolDescriptors(doravirine)

# Next, we can use pymatgen software to collect 74 new descriptors:

In [5]:
from pymatgen.ext.matproj import MPRester

In [6]:
#we will always take the first record
from pymatgen.ext.matproj import MPRester
with MPRester(api_key='px27m7fBcI17PKBcjzdb6AtN8j7v9SQH') as m:
     results = m.summary.search(formula="NaCl")

Retrieving SummaryDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
descriptors_pymatgen = []
print(len(list(results[0])))
print()
for i in range(len(list(results[0]))):
    descriptors_pymatgen.append(list(results[0])[i][0])
    print(list(results[0])[i][0])

74

builder_meta
nsites
elements
nelements
composition
composition_reduced
formula_pretty
formula_anonymous
chemsys
volume
density
density_atomic
symmetry
property_name
material_id
deprecated
deprecation_reasons
last_updated
origins
warnings
structure
task_ids
uncorrected_energy_per_atom
energy_per_atom
formation_energy_per_atom
energy_above_hull
is_stable
equilibrium_reaction_energy_per_atom
decomposes_to
xas
grain_boundaries
band_gap
cbm
vbm
efermi
is_gap_direct
is_metal
es_source_calc_id
bandstructure
dos
dos_energy_up
dos_energy_down
is_magnetic
ordering
total_magnetization
total_magnetization_normalized_vol
total_magnetization_normalized_formula_units
num_magnetic_sites
num_unique_magnetic_sites
types_of_magnetic_species
k_voigt
k_reuss
k_vrh
g_voigt
g_reuss
g_vrh
universal_anisotropy
homogeneous_poisson
e_total
e_ionic
e_electronic
n
e_ij_max
weighted_surface_energy_EV_PER_ANG2
weighted_surface_energy
weighted_work_function
surface_anisotropy
shape_factor
has_reconstructed
possib

In [8]:
#create a DataFrame that will include all data about descriptors
df_pymatgen = pd.DataFrame(columns=descriptors_pymatgen)

In [9]:
df_pymatgen.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   builder_meta                                  0 non-null      object
 1   nsites                                        0 non-null      object
 2   elements                                      0 non-null      object
 3   nelements                                     0 non-null      object
 4   composition                                   0 non-null      object
 5   composition_reduced                           0 non-null      object
 6   formula_pretty                                0 non-null      object
 7   formula_anonymous                             0 non-null      object
 8   chemsys                                       0 non-null      object
 9   volume                                        0 non-null      object
 10  density            

In [ ]:
# enumeration of all names in the DataFrame. Calculated for this name descriptors is inserting in the df_pcp Dataframe. 
#If this name cannot be used for calculating descriptors, nan row is incerting
df_nan = pd.DataFrame(columns=descriptors_pymatgen, index=[0])
for i,_ in df_initial.iterrows():
    try:
        df_new_row = pd.DataFrame(columns=descriptors_pymatgen, index=[0])
        with MPRester(api_key='px27m7fBcI17PKBcjzdb6AtN8j7v9SQH') as m:
            results = m.summary.search(formula=df_initial.Name[i]) 
        for i in range(len(list(results[0]))):
            df_new_row.at[0, list(results[0])[i][0]] = list(results[0])[i][1]
        df_pymatgen = pd.concat([df_pymatgen, df_new_row])
    except:
        df_pymatgen = pd.concat([df_pymatgen, df_nan])

Retrieving SummaryDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/367 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/9 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/790 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/790 [00:00<?, ?it/s]

...

Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

Retrieving SummaryDoc documents: 0it [00:00, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/790 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

In [62]:
# enumeration of all names in the DataFrame. Calculated for this name descriptors is inserting in the df_pcp Dataframe. 
#If this name cannot be used for calculating descriptors, nan row is incerting
df_nan = pd.DataFrame(columns=descriptors_pymatgen, index=[0]) as as 
k = 0
for i,_ in df_initial.iterrows():
    k += 1
    if k < 10:
        try:
            df_new_row = pd.DataFrame(columns=descriptors_pymatgen, index=[0])
            with MPRester(api_key='px27m7fBcI17PKBcjzdb6AtN8j7v9SQH') as m:
                results = m.summary.search(formula=df_initial.Name[i]) 
            for i in range(len(list(results[0]))):
                df_new_row.at[0, list(results[0])[i][0]] = list(results[0])[i][1]
            df_pymatgen = pd.concat([df_pymatgen, df_new_row])
        except:
            df_pymatgen = pd.concat([df_pymatgen, df_nan])
    else:
        break

Retrieving SummaryDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/367 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/9 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/790 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/790 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/322 [00:00<?, ?it/s]

In [63]:
df_pymatgen

,builder_meta,nsites,elements,nelements,composition,composition_reduced,formula_pretty,formula_anonymous,chemsys,volume,...,weighted_surface_energy,weighted_work_function,surface_anisotropy,shape_factor,has_reconstructed,possible_species,has_props,theoretical,database_IDs,fields_not_requested
0,emmet_version='0.38.6' pymatgen_version='2022....,12,"[Cl, Na]",2,"(Na, Cl)","(Na, Cl)",NaCl,AB,Cl-Na,284.082319,...,None,None,None,None,None,"[Na+, Cl-]","[HasProps.oxi_states, HasProps.thermo, HasProp...",True,{},[]
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,emmet_version='0.38.6' pymatgen_version='2022....,16,"[F, H, Mg, O]",4,"(Mg, H, O, F)","(Mg, H, O, F)",MgHOF,ABCD,F-H-Mg-O,151.014639,...,None,None,None,None,None,"[F-, Mg2+, H+, O2-]","[HasProps.oxi_states, HasProps.materials, HasP...",False,{icsd: ['icsd-186501']},[]
0,emmet_version='0.38.6' pymatgen_version='2022....,4,"[Cu, O]",2,"(Cu, O)","(Cu, O)",CuO,AB,Cu-O,44.21362,...,None,None,None,None,None,"[O2-, Cu2+]","[HasProps.thermo, HasProps.materials, HasProps...",True,{},[]
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,emmet_version='0.38.6' pymatgen_version='2022....,4,[Si],1,(Si),(Si),Si,A,Si,85.03542,...,None,None,None,None,None,[],"[HasProps.electronic_structure, HasProps.dos, ...",True,{icsd: ['icsd-181908']},[]
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,emmet_version='0.38.6' pymatgen_version='2022....,4,[Si],1,(Si),(Si),Si,A,Si,85.03542,...,None,None,None,None,None,[],"[HasProps.electronic_structure, HasProps.dos, ...",True,{icsd: ['icsd-181908']},[]
0,emmet_version='0.38.6' pymatgen_version='2022....,36,"[O, Si]",2,"(Si, O)","(Si, O)",SiO2,AB2,O-Si,611.879244,...,None,None,None,None,None,"[Si4+, O2-]","[HasProps.substrates, HasProps.dielectric, Has...",False,"{icsd: ['icsd-69114', 'icsd-162441', 'icsd-611...",[]


In [56]:
df_new_row = pd.DataFrame(columns=descriptors_pymatgen, index=[0])
with MPRester(api_key='px27m7fBcI17PKBcjzdb6AtN8j7v9SQH') as m:
     results = m.summary.search(formula="NaCl") 
for i in range(len(list(results[0]))):
    df_new_row.at[0, list(results[0])[i][0]] = list(results[0])[i][1]

Retrieving SummaryDoc documents:   0%|          | 0/3 [00:00<?, ?it/s]

In [57]:
df_new_row

,builder_meta,nsites,elements,nelements,composition,composition_reduced,formula_pretty,formula_anonymous,chemsys,volume,...,weighted_surface_energy,weighted_work_function,surface_anisotropy,shape_factor,has_reconstructed,possible_species,has_props,theoretical,database_IDs,fields_not_requested
0,emmet_version='0.38.6' pymatgen_version='2022....,12,"[Cl, Na]",2,"(Na, Cl)","(Na, Cl)",NaCl,AB,Cl-Na,284.082319,...,None,None,None,None,None,"[Na+, Cl-]","[HasProps.oxi_states, HasProps.thermo, HasProp...",True,{},[]


# Try PaDelPy

In [5]:
%pip install padelpy

Note: you may need to restart the kernel to use updated packages.


In [6]:
from padelpy import from_smiles
import pubchempy as pcp
import pandas as pd

In [7]:
# calculate molecular descriptors for propane
descriptors_PaDELPy = list(from_smiles('CCC'))
descriptors_values = list(from_smiles('CCC').values())

TypeError: expected string or bytes-like object

In [ ]:
#create a DataFrame that will include all data about descriptors
df_PaDELPy = pd.DataFrame(columns=descriptors_PaDELPy)

In [ ]:
# enumeration of all names in the DataFrame. Calculated for this name descriptors is inserting in the df_rdkit Dataframe. 
#If this name cannot be used for calculating descriptors, nan row is incerting
df_nan = pd.DataFrame(columns=descriptors_PaDELPy, index=[0])
for i,_ in df_initial.iterrows():
    try:
        df_new_row = pd.DataFrame(columns=descriptors_PaDELPy, index=[0])
        compound = pcp.get_compounds(df_initial.Name[i], 'name')
        smile = compound[0].canonical_smiles
        df_PaDELPy = pd.concat([df_PaDELPy, pd.DataFrame([from_smiles(smile)])])
    except:
        df_PaDELPy = pd.concat([df_PaDELPy, df_nan])